# 使用AgentExecutor构建代理（旧版）
:::重要本节将介绍如何使用传统的LangChain AgentExecutor进行构建。虽然这些工具适合入门，但到了一定阶段后，您可能会需要它们无法提供的灵活性和控制。对于更高级的智能体开发，我们推荐您查阅[LangGraph智能体](/docs/concepts/architecture/#langgraph)或[迁移指南](/docs/how_to/migrate_agent/)。:::
语言模型本身无法采取行动——它们仅输出文本。LangChain 的一个重要应用场景是构建 **[智能代理](/docs/concepts/agents/)**。代理（Agents）是一种将大语言模型（LLM）作为推理引擎的系统，用于确定需要执行哪些操作以及这些操作的输入内容。这些行动的结果随后可以反馈给代理，由它决定是否需要采取更多行动，或者是否可以结束。
在本教程中，我们将构建一个能够与多种工具交互的智能体：其中一个工具是本地数据库，另一个是搜索引擎。您将能够向该智能体提问，观察它调用工具的过程，并与它进行对话交流。
## 概念
我们将涵盖的概念包括：- 使用[语言模型](/docs/concepts/chat_models)，特别是其工具调用能力- 创建一个[检索器](/docs/concepts/retrievers)，用于向我们的智能体公开特定信息- 使用搜索[工具](/docs/concepts/tools)在线查找信息- [`聊天历史`](/docs/concepts/chat_history)，使聊天机器人能够"记住"过去的互动，并在回答后续问题时将其考虑在内。- 使用 [LangSmith](https://docs.smith.langchain.com/) 调试和追踪您的应用程序
## 安装设置
### Jupyter Notebook
本指南（以及文档中的大多数其他指南）使用 [Jupyter Notebooks](https://jupyter.org/)，并假设读者也在使用该工具。Jupyter Notebooks 非常适合学习如何构建 LLM 系统，因为过程中常会出现意外情况（例如输出不符预期、API 服务中断等），而通过交互式环境学习指南能帮助您更深入地理解这些内容。
本教程及其他教程或许在 Jupyter notebook 中运行最为便捷。有关安装说明，请参阅[此处](https://jupyter.org/install)。
### 安装
要安装LangChain，请运行：
import Tabs from '@theme/Tabs';import TabItem from '@theme/TabItem';import CodeBlock from "@theme/CodeBlock";
<Tabs>
  <TabItem value="pip" label="Pip" default>
<CodeBlock language="bash">pip install langchain</CodeBlock>  </TabItem>
  <TabItem value="conda" label="Conda">
<CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>  </TabItem>
</Tabs>



更多详情，请参阅我们的[安装指南](/docs/how_to/installation)。
### LangSmith
使用LangChain构建的许多应用程序将包含多个步骤，涉及多次LLM调用。随着这些应用变得越来越复杂，能够检查链或代理内部的具体运行情况变得至关重要。最佳方式是通过 [LangSmith](https://smith.langchain.com) 实现。
在以上链接完成注册后，请确保设置环境变量以开始记录追踪数据：
```shellexport LANGSMITH_TRACING="true"export LANGSMITH_API_KEY="..."好的,我将按照要求进行翻译,只输出翻译后的中文markdown内容:

# 欢迎使用翻译助手

这是一个标准的markdown格式文档示例,包含以下元素:

## 二级标题

- 项目符号列表项1
- 项目符号列表项2
- 项目符号列表项3

### 三级标题

1. 有序列表项1
2. 有序列表项2
3. 有序列表项3

**粗体文本** 和 *斜体文本*

[链接示例](https://example.com)

> 引用文本块

`行内代码`

```python
# 代码块示例
def hello():
    print("Hello World!")
```
或者，如果在笔记本中，您可以通过以下方式设置：
```python```markdown
import getpass
```import os
os.environ["LANGSMITH_TRACING"] = "true"os.environ["LANGSMITH_API_KEY"] = getpass.getpass()好的,我将严格按照您的要求进行翻译,确保markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个标准的markdown格式文档翻译示例。

## 主要功能

- 保持原始markdown格式
- 准确传达原文意思
- 提供自然流畅的中文表达

### 注意事项

1. 不添加额外说明文字
2. 保持标题层级结构
3. 列表格式完全对应

**强调内容** 和 _斜体内容_ 也会被正确转换。

> 引用块也会被妥善处理

[链接文本](https://example.com) 和 `代码片段` 都会保留原格式。

## 定义工具
我们首先需要创建要使用的工具。我们将使用两种工具：[Tavily](/docs/integrations/tools/tavily_search)（用于在线搜索）以及一个基于本地索引构建的检索器
### [Tavily](/docs/integrations/tools/tavily_search)
LangChain 内置了一个工具，可轻松将 Tavily 搜索引擎作为工具使用。请注意，这需要一个API密钥——他们提供免费层级服务，但如果您没有密钥或不想创建，随时可以跳过此步骤。
创建API密钥后，您需要将其导出为：
```bashexport TAVILY_API_KEY="..."好的,我将按照要求进行翻译,确保输出标准的markdown格式内容,不显示任何额外标记。以下是翻译结果:

# 欢迎使用翻译助手

## 功能特点

1. **多语言支持**: 支持中英文互译
2. **格式保留**: 保持原始文档的markdown格式
3. **准确翻译**: 提供专业准确的翻译结果

## 使用说明

1. 输入需要翻译的英文文本
2. 系统会自动识别并翻译成中文
3. 翻译结果将保持原有的markdown格式

> 注意: 请确保输入的英文文本是标准的markdown格式

## 示例

```python
def hello_world():
    print("Hello, World!")
```

上面的代码示例会输出"Hello, World!"

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [6]:
search = TavilySearchResults(max_results=2)

In [7]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714000492, 'localtime': '2024-04-24 16:14'}, 'current': {'last_updated_epoch': 1713999600, 'last_updated': '2024-04-24 16:00', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 100, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 14.8, 'gust_kph': 23.8}}"},
 {'url': 'https://www.weathertab.com/en/c/e/04/united-states/california/san-francisco/',
  'content': 'San Francisco Weather Forecast for Apr 2024 - Risk of Rain Graph. R

### 检索器
我们还将基于自有数据创建一个检索器。如需深入了解每个步骤的详细说明，请参阅[本教程](/docs/tutorials/rag)。

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [9]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='# The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from \'langsmith\';import { runOnDataset } from \'langchain/smith\';import { EvaluationResult } from \'langsmith/evaluation\';const client = new Client();// Define dataset: these are your test casesconst datasetName = "Sample Dataset";const dataset = await client.createDataset(datasetName, {    description: "A sample dataset in LangSmith."});await client.createExamples({    inputs: [        { postfix: "to LangSmith" },        { postfix: "to Evaluations in LangSmith" },    ],    outputs: [        { output: "Welcome to LangSmith" },        { output: "Welcome to Evaluations in LangSmith" },    ],    datasetId: dataset.id,});// Define your evaluatorconst exactMatch = async ({ run, example }: {

既然我们已经填充了用于检索的索引，现在可以轻松地将其转换为工具（这是代理程序正确使用所需的格式）。

In [10]:
from langchain.tools.retriever import create_retriever_tool

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### 工具
既然我们已经创建了这两者，现在可以列出一个下游将使用的工具清单。

In [12]:
tools = [search, retriever_tool]

## 使用语言模型
接下来，我们将学习如何通过调用工具来使用语言模型。LangChain 支持多种可互换的语言模型——请从下方选择您想使用的模型！
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />


In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

你可以通过传入消息列表来调用语言模型。默认情况下，响应是一个 `content` 字符串。

In [13]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

我们现在可以看到启用该模型进行工具调用的效果。为了启用这一功能，我们使用`.bind_tools`方法让语言模型了解这些工具。

In [14]:
model_with_tools = model.bind_tools(tools)

我们现在可以调用模型了。首先用一个普通消息来调用它，看看它是如何响应的。我们可以同时查看 `content` 字段和 `tool_calls` 字段。

In [18]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


现在，让我们尝试用一些预期会调用工具的输入来调用它。

In [19]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4HteVahXkRAkWjp6dGXryKZX'}]


我们可以看到现在没有任何内容，但有一个工具调用！它希望我们调用Tavily搜索工具。
这还不是在调用那个工具——它只是告诉我们要这么做。为了真正调用它，我们需要创建我们的代理。

## 创建代理
既然我们已经定义了工具和大语言模型（LLM），现在可以创建智能体了。我们将使用一种工具调用型智能体——如需了解更多关于此类智能体及其他选项的信息，请参阅[本指南](/docs/concepts/agents/)。
我们可以首先选择想要用来引导智能体的提示词。
若想查看此提示内容并拥有LangSmith访问权限，可前往：
[https://smith.langchain.com/hub/hwchase17/openai-functions-agent](https://smith.langchain.com/hub/hwchase17/openai-functions-agent)

In [20]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

现在，我们可以用LLM（大语言模型）、提示词和工具来初始化智能体。该智能体负责接收输入并决定采取何种行动。关键在于，智能体本身并不执行这些行动——实际执行由AgentExecutor完成（下一步骤）。若想深入了解这些组件的设计理念，请参阅我们的[概念指南](/docs/concepts/agents)。
请注意，我们传入的是 `model`，而非 `model_with_tools`。这是因为 `create_tool_calling_agent` 会在底层自动为我们调用 `.bind_tools` 方法。

In [23]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

最后，我们将智能体（大脑）与工具整合到AgentExecutor中（该执行器会反复调用智能体并执行工具）。

In [24]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

## 运行代理
我们现在可以在一些查询上运行代理了！请注意，目前这些都是**无状态**查询（它不会记住之前的交互）。
首先，让我们看看它在无需调用工具时的反应：

In [25]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

为了准确了解底层运行机制（并确保其未调用工具），我们可以查看 [LangSmith 追踪记录](https://smith.langchain.com/public/8441812b-94ce-4832-93ec-e1114214553a/r)。
现在让我们通过一个示例来尝试，这个示例应该会调用检索器。

In [26]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform that aids in building production-grade Language Learning Model (LLM) applications. It can assist with testing in several ways:\n\n1. **Monitoring and Evaluation**: LangSmith allows close monitoring and evaluation of your application. This helps you to ensure the quality of your application and deploy it with confidence.\n\n2. **Tracing**: LangSmith has tracing capabilities that can be beneficial for debugging and understanding the behavior of your application.\n\n3. **Evaluation Capabilities**: LangSmith has built-in tools for evaluating the performance of your LLM. \n\n4. **Prompt Hub**: This is a prompt management tool built into LangSmith that can help in testing different prompts and their responses.\n\nPlease note that to use LangSmith, you would need to install it and create an API key. The platform offers Python and Typescript SDKs for utilization. It works independently and does not require th

让我们查看一下 [LangSmith 追踪记录](https://smith.langchain.com/public/762153f6-14d4-4c98-8659-82650f860c62/r) 以确保它确实调用了该功能。
现在让我们尝试一个需要调用搜索工具的例子：

In [27]:
agent_executor.invoke({"input": "whats the weather in sf?"})

{'input': 'whats the weather in sf?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 16.1°C (61.0°F). The wind is coming from the WNW at a speed of 10.5 mph. The humidity is at 67%. [source](https://www.weatherapi.com/)'}

我们可以查看 [LangSmith 追踪记录](https://smith.langchain.com/public/36df5b1a-9a0b-4185-bae2-964e1d53c665/r) 来确认它是否有效调用了搜索工具。

## 添加至内存
如前所述，该代理是无状态的。这意味着它不会记住之前的交互。为了赋予其记忆能力，我们需要传入先前的 `chat_history`。注意：由于我们使用的提示词要求，该变量必须命名为 `chat_history`。如果使用不同的提示词，我们可以更改变量名称。

In [28]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})

{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [29]:
from langchain_core.messages import AIMessage, HumanMessage

In [30]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)

{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I assist you further?'}

如果我们想要自动跟踪这些消息，可以使用 `RunnableWithMessageHistory` 进行封装。有关如何使用此功能的更多信息，请参阅[本指南](/docs/how_to/message_history)。

In [36]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

由于我们有多个输入，需要明确以下两点：
- `input_messages_key`: 用于添加到对话历史中的输入键。- `history_messages_key`: 用于添加已加载消息的键名。

In [37]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [38]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [39]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

示例 LangSmith 跟踪记录：https://smith.langchain.com/public/98c8d162-60ae-4493-aa9f-992d87bd0429/r

## 结论
搞定！在这个快速入门教程中，我们介绍了如何创建一个简单的智能体。智能体是个复杂的主题，还有很多内容值得深入学习！
:::重要本节介绍了如何使用LangChain智能体进行构建。它们适合入门学习，但超过一定程度后，您可能会需要它们所不具备的灵活性和控制力。要开发更高级的智能体，我们建议您查阅[LangGraph](/docs/concepts/architecture/#langgraph)。:::
如果你想继续使用LangChain代理，以下是一些不错的高级指南：
- [如何使用 LangGraph 内置版本的 `AgentExecutor`](/docs/how_to/migrate_agent)- [如何创建自定义代理](https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/)- [如何从代理流式传输响应](https://python.langchain.com/v0.1/docs/modules/agents/how_to/streaming/)- [如何从智能体返回结构化输出](https://python.langchain.com/v0.1/docs/modules/agents/how_to/agent_structured/)